# Pipeline 2 Imports

In [ ]:
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 876.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch
import torchaudio
import torchaudio.functional as F

# Pipeline 2 Functions

[STFT Window Functions](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C2/C2_STFT-Window.html)
- Hann Window => raised cosine window, drops smoothly to 0 at section boundaries
  - BUT smears some frequencies = reduction of ripple artifacts

[Types of Window Functions](https://en.wikipedia.org/wiki/Window_function)

In [1]:
# BASE Short-Time Fourier Transform (STFT) Computation
def compute_stft(waveform, window, sample_rate=16000):
    """
    Compute STFT with specified parameters.

    Parameters:
        waveform (Tensor): Audio waveform, shape (B, N)
        window (str): Window function type
        sample_rate (int): Sampling rate

    Return tensor: STFT magnitude, shape (B, 257, 498)
    """
    n_fft = 512
    # window size = win_length
    win_length = int(0.025 * sample_rate)  # 25 ms → 400 samples
    hop_length = 160

    # Different types of window functions
    if window == "Rectangular":
      window = torch.ones(win_length).to(waveform.device)
    elif window == "Triangle":
      window = torch.trianglular_window(win_length).to(waveform.device)
    elif window == "Hann":
      window = torch.hann_window(win_length).to(waveform.device)
    elif window == "Tukey":
      window = torch.tukey_window(win_length).to(waveform.device)

    # Compute stft, output shape: (B, 257, 498)
    stft = torch.stft(
        waveform, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
        window=window, return_complex=True
    )

    # Magnitude (abs) of complex STFT
    stft_mag = torch.abs(stft)
    return stft_mag

In [ ]:
# MEL Spectrogram Conversion
def stft_to_mel(stft_mag, sample_rate=16000, n_mels=80):
    """
    Convert STFT magnitude to Mel Spectrogram.

    Parameters:
        stft_mag (Tensor): Tensor of shape (B, 257, 498)
        sample_rate (int): Sampling rate of the original signal
        n_mels (int): Number of Mel filters

    Returns tensor: Mel spectrogram of shape (B, 498, 80)
    """
    # Create Mel filter bank, (257, 80)
    mel_filter_bank = torchaudio.functional.create_fb_matrix(
        n_freqs=257, f_min=0.0, f_max=sample_rate / 2.0,
        n_mels=n_mels, sample_rate=sample_rate
    )

    # Apply the Mel filter bank, transpose to time-first format
    mel_spec = torch.matmul(stft_mag.transpose(1, 2), mel_filter_bank)
    # (B, 498, 80)
    return mel_spec

In [ ]:
# MFCC Extraction
def compute_mfcc_stack(mel_spec):
    """
    Compute MFCCs, deltas, and delta-deltas from a Mel spectrogram.

    Parameters:
        mel_spec (Tensor): Input Mel spectrogram of shape (B, 80, T)

    Returns tensor: Final stacked features of shape (B, T, 39)
    """
    # Batch, Frequency, Time
    B, M, T = mel_spec.shape
    # (B, 80, 498)

    # DCT (Discrete Cosine Transform) Type-II over frequency axis
    mfcc = F.compute_dct(mel_spec, norm="ortho", type=2)
    # (B, 80, T)

    # Keep first 13 MFCCs (most informative)
    mfcc = mfcc[:, :13, :]
    # (B, 13, T)

    # Delta (first derivative)
    delta = F.compute_deltas(mfcc)
    # (B, 13, T)

    # Delta-Delta (second derivative)
    delta_delta = F.compute_deltas(delta)
    # (B, 13, T)

    # Stack all coefficcients (MFCC, delta, delta-delta)
    mfcc_stack = torch.cat([mfcc, delta, delta_delta], dim=1)
    # (B, 39, T)

    # Step 6: Transpose to time-first format
    return mfcc_stack.transpose(1, 2)
    # (B, 498, 39)